In [ ]:
import os
from picai_eval import Metrics
from picai_eval import evaluate_folder
from report_guided_annotation import extract_lesion_candidates
import shutil
import matplotlib.pyplot as plt
from sklearn.metrics import PrecisionRecallDisplay, RocCurveDisplay
import seaborn as sns


In [ ]:
folds_path = ['/path/to/nnUNet_v2/nnUNet_results/Dataset104_PANORAMA_baseline_PDAC_Detection/nnUNetTrainer_Loss_CE_checkpoints__nnUNetPlans__3d_fullres/fold_0',
              '/path/to/nnUNet_v2/nnUNet_results/Dataset104_PANORAMA_baseline_PDAC_Detection/nnUNetTrainer_Loss_CE_checkpoints__nnUNetPlans__3d_fullres/fold_1',
              '/path/to/nnUNet_v2/nnUNet_results/Dataset104_PANORAMA_baseline_PDAC_Detection/nnUNetTrainer_Loss_CE_checkpoints__nnUNetPlans__3d_fullres/fold_2',
              '/path/to/nnUNet_v2/nnUNet_results/Dataset104_PANORAMA_baseline_PDAC_Detection/nnUNetTrainer_Loss_CE_checkpoints__nnUNetPlans__3d_fullres/fold_3',
              '/path/to/nnUNet_v2/nnUNet_results/Dataset104_PANORAMA_baseline_PDAC_Detection/nnUNetTrainer_Loss_CE_checkpoints__nnUNetPlans__3d_fullres/fold_4' ]


In [ ]:
#Gather results per checkpoints for each fold
checkpoints = list(range(50, 951, 50))
in_dir_annot = '/path/to/nnUNet_v2/nnUNet_raw/Dataset104_PANORAMA_baseline_PDAC_Detection/labelsTr'
results_dict = {}

for fold_dir in folds_path:
    fold = fold_dir.split('/')[-1]
    print('Analysing fold', fold)
    metrics = []
    weighted_auroc_ap = {}
    auroc_list = []
    ap_list = []
    results_fold = {}
    for check in checkpoints:
        metrics_file = os.path.join(fold_dir, f'metrics_check_{check}.json')
        if os.path.exists(metrics_file):
            try:
                metric = Metrics(metrics_file)
                metrics.append(metric)
                auroc_list.append(metric.auroc)
                ap_list.append(metric.AP)
                weighted_auroc_ap[check] =  0.5 * (metric.auroc + metric.AP)
            except Exception as e:
                print(f"Error processing metrics for checkpoint {check} in fold {fold}: {e}")

    results_fold['metrics'] = metrics
    results_fold['weighted_auroc_ap'] = weighted_auroc_ap
    results_fold['auroc_list'] = auroc_list
    results_fold['ap_list'] = ap_list
    results_dict[fold] = results_fold


In [ ]:
# Number of folds to determine the grid of subplots
num_folds = len(results_dict)
cols = 2  # Number of columns in subplot grid
rows = (num_folds + 1) // cols  # Calculate rows needed, ensure there's enough space

plt.figure(figsize=(15, 6 * rows))  # Adjust overall figure size accordingly

# Counter for subplot index
subplot_idx = 1

for fold, data in results_dict.items():
    checkpoints = list(data['weighted_auroc_ap'].keys())
    auroc_scores = data['auroc_list']
    ap_scores = data['ap_list']
    weighted_scores = [data['weighted_auroc_ap'][chk] for chk in checkpoints]  # Extract weighted scores in order

    # Finding the indices of the maximum weighted score and AP
    max_weighted_index = weighted_scores.index(max(weighted_scores))
    max_ap_index = ap_scores.index(max(ap_scores))
    max_auroc_index = auroc_scores.index(max(auroc_scores))

    # Create a subplot for each fold
    ax = plt.subplot(rows, cols, subplot_idx)
    ax.plot(checkpoints, auroc_scores, label='AUROC', marker='o', linestyle='-', color='blue')
    ax.plot(checkpoints[max_auroc_index], auroc_scores[max_auroc_index], 'b*', markersize=12, label='Max AUROC')

    ax.plot(checkpoints, ap_scores, label='AP', marker='x', linestyle='--', color='green')
    ax.plot(checkpoints[max_ap_index], ap_scores[max_ap_index], 'g*', markersize=12, label='Max AP')

    ax.plot(checkpoints, weighted_scores, label='Weighted AUROC+AP', marker='s', linestyle=':', color='red')
    # Highlight the max points with a star marker
    ax.plot(checkpoints[max_weighted_index], weighted_scores[max_weighted_index], 'r*', markersize=12, label='Max Weighted')

    ax.set_title(f'Fold {fold}')
    ax.set_xlabel('Checkpoint')
    ax.set_ylabel('Score')
    ax.legend()
    ax.set_xticks(checkpoints)  # Set all checkpoints as ticks
    ax.set_xticklabels(checkpoints, rotation=45)  # Rotate labels to fit and improve readability

    # Increment the subplot index
    subplot_idx += 1

# Adjust layout to prevent overlapping
plt.tight_layout()
plt.show()


In [ ]:
for fold, data in results_dict.items():
    fold_path = os.path.join('/path/to/nnUNet_v2/nnUNet_results/Dataset104_PANORAMA_baseline_PDAC_Detection/nnUNetTrainer_Loss_CE_checkpoints__nnUNetPlans__3d_fullres', fold)
    checkpoints = list(data['weighted_auroc_ap'].keys())
    weighted_scores = [data['weighted_auroc_ap'][chk] for chk in checkpoints]
    
    # Determine the best checkpoint by maximum weighted AUROC+AP
    best_idx = weighted_scores.index(max(weighted_scores))
    best_checkpoint = checkpoints[best_idx]
    print(f'Best checkpoint for fold {fold} is {best_checkpoint}')

    # Construct the source file path for the best checkpoint
    source_file = os.path.join(fold_path, f'checkpoint_{best_checkpoint}.pth')

    # Construct the destination file path for the best checkpoint
    destination_file = os.path.join(fold_path, 'checkpoint_best_panorama.pth')

    # Copy the file to the new destination with the new name
    if os.path.exists(source_file) and os.path.exists(destination_file)==False:
        shutil.copy(source_file, destination_file)
        print(f"Copied best checkpoint for {fold} from {source_file} to {destination_file}")
    else:
        print(f"File not found: {source_file} - Checkpoint {best_checkpoint} for fold {fold}")


In [ ]:
metrics_path = ['/path/to/nnUNet_v2/nnUNet_results/Dataset104_PANORAMA_baseline_PDAC_Detection/nnUNetTrainer_Loss_CE_checkpoints__nnUNetPlans__3d_fullres/fold_0/metrics_check_950.json',
                '/path/to/nnUNet_v2/nnUNet_results/Dataset104_PANORAMA_baseline_PDAC_Detection/nnUNetTrainer_Loss_CE_checkpoints__nnUNetPlans__3d_fullres/fold_1/metrics_check_950.json',
                '/path/to/nnUNet_v2/nnUNet_results/Dataset104_PANORAMA_baseline_PDAC_Detection/nnUNetTrainer_Loss_CE_checkpoints__nnUNetPlans__3d_fullres/fold_2/metrics_check_750.json',
                '/path/to/nnUNet_v2/nnUNet_results/Dataset104_PANORAMA_baseline_PDAC_Detection/nnUNetTrainer_Loss_CE_checkpoints__nnUNetPlans__3d_fullres/fold_3/metrics_check_950.json',
                '/path/to/nnUNet_v2/nnUNet_results/Dataset104_PANORAMA_baseline_PDAC_Detection/nnUNetTrainer_Loss_CE_checkpoints__nnUNetPlans__3d_fullres/fold_4/metrics_check_950.json']


num_folds = len(metrics_path)
cols = 2  # Number of columns in subplot grid
rows = (num_folds + 1) // cols  # Calculate rows needed, ensure there's enough space

plt.figure(figsize=(15, 6 * rows))  # Adjust overall figure size accordingly

subplot_idx = 1
for metrics_file in metrics_path:
    metrics = Metrics(metrics_file)
    tpr = metrics.case_TPR
    fpr = metrics.case_FPR
    auroc = metrics.auroc

    disp = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=auroc)
    ax = plt.subplot(rows, cols, subplot_idx)
    disp.plot(ax=ax)
    ax.set_title(f'Fold {subplot_idx - 1}')
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.legend(loc="lower right")

    subplot_idx += 1

# Adjust layout to prevent overlapping
plt.tight_layout()
plt.show()

In [ ]:
num_folds = len(metrics_path)
cols = 2  # Number of columns in subplot grid
rows = (num_folds + 1) // cols  # Calculate rows needed, ensure there's enough space

plt.figure(figsize=(15, 6 * rows))  # Adjust overall figure size accordingly

subplot_idx = 1
for metrics_file in metrics_path:
    metrics = Metrics(metrics_file)
    prediction_values = metrics.case_pred

    ax = plt.subplot(rows, cols, subplot_idx)
    sns.kdeplot(prediction_values, ax=ax, fill=True, clip=[0, 1])  # Clip the KDE plot within [0, 1]
    ax.set_title(f'Fold {subplot_idx - 1} Prediction Distribution')
    ax.set_xlabel('Prediction Scores')
    ax.set_ylabel('Density')

    subplot_idx += 1

# Adjust layout to prevent overlapping
plt.tight_layout()
plt.show()